# Upgrading our MNIST Network

In [1]:
using MLDatasets
train_x, train_y = MNIST.traindata()
test_x,  test_y  = MNIST.testdata();

(images, labels) = (reshape(train_x[:,:,1:1000], (1000, 28*28)), train_y[1:1000])
one_hot_labels = zeros(length(labels), 10)
for (i,l) in enumerate(labels)
    one_hot_labels[i,l+1] = 1.0
end
labels = one_hot_labels

test_images = reshape(test_x, (size(test_x)[end], 28*28))
test_labels = zeros((size(test_x)[end],10))

for (i,l) in enumerate(test_y)
    test_labels[i,l+1] = 1.0
end

using Random
Random.seed!(1)

tanh2deriv(output) = 1 - output^2

function softmax(x)
    temp = exp.(x)
    return temp ./ sum(temp)
end

function softmax(x):
    temp = np.exp(x)
    return temp ./ sum(temp, dims=2)
end

alpha, iterations, hidden_size = (2, 300, 100)
pixels_per_image, num_labels = (784, 10)
batch_size = 100

weights_0_1 = 0.2 .* rand(pixels_per_image,hidden_size) .- 0.1
weights_1_2 = 0.2 .* rand(hidden_size,num_labels) .- 0.1

for j = 1:iterations
    Error, Correct_cnt = (0.0, 0)
    for i = 1:batch_size:size(images)[1]-batch_size
        batch_start, batch_end = i, i+batch_size-1
        layer_0 = images[batch_start:batch_end,:]
        layer_1 = tanh.(layer_0 * weights_0_1)
        dropout_mask = bitrand(size(layer_1))
        layer_1 .*= (dropout_mask .* 2)
        layer_2 = softmax(layer_1 * weights_1_2)
        
        Error += sum((labels[batch_start:batch_end,:] .- layer_2) .^ 2)
        
        for k=1:batch_size
            Correct_cnt += Int(argmax(layer_2[k,:]) == argmax(labels[batch_start+k-1,:]))
            layer_2_delta = (labels[batch_start:batch_end,:] .- layer_2) ./batch_size
            layer_1_delta = (layer_2_delta * weights_1_2') .* relu2deriv.(layer_1)

            layer_1_delta .*= dropout_mask

            weights_1_2 += alpha .* layer_1 * layer_2_delta
            
            println(size(layer_1), size(layer_2_delta))
            @assert false
            weights_0_1 += alpha .* layer_0' * layer_1_delta
        end
    end
        
    if (j % 10 == 0)
        test_Error, test_Correct_cnt = (0.0, 0)
        for i = 1:size(test_images)[1]

            layer_0 = test_images[i,:]
            layer_1 = relu.(layer_0' * weights_0_1)
            layer_2 = layer_1 * weights_1_2

            test_Error += sum((test_labels[i,:]' .- layer_2) .^ 2)
            test_Correct_cnt += Int(argmax(layer_2) == argmax(test_labels[i,:]'))
        end
        println("I: $(j) Train error: $(Error/size(images)[1])% Train accuracy: $(Correct_cnt/size(images)[1]) Test-Err:: $(test_Error/size(test_images)[1])% Test-Acc:: $(test_Correct_cnt/size(test_images)[1])")
    end
end

